In [ ]:
#Extracting "rs" containing XML files from Pubmed OAI set.
import os
import sys
import shutil
directory = "/home/werner/Desktop/Source/articles.I-N/" #example subset
filelisting = os.walk(directory)
totalFiles = 0
rsFiles = 0
for root, dirs, files in filelisting:   
    for file in files:
        totalFiles += 1
        breakTest = 0
        fileone = open(root + "/" + file)
        if breakTest == 1:
            break
        for line in fileone:
            line1 = line.split()
            if breakTest == 1:
                break
            for word in line1:
                if (word.startswith('rs') or (not word[0].isalnum() and "rs" in word[1:3])): # search clause NEW CLAUSE ADDED
                    shutil.copy(os.path.join(root,file), "/home/werner/Desktop/Destination/" + file)
                    breakTest = 1
                    rsFiles += 1
                if breakTest == 1:
                    break
        fileone.close()
print (totalFiles, rsFiles)

In [12]:
#Create flat file database
import xml.etree.ElementTree as ET
import os
import json
directory = "/home/werner/Desktop/Destination/"
filelisting = os.walk(directory)
rslist =[]
for root, dirs, files in filelisting:   
    for file in files:
        email = "not available" #reset
        pmid = "not available" #reset
        year = "not available" #reset
        doi = "not available" #reset
        kwds = [] #reset
        tree = ET.parse(root + "/" + file)
        for node in tree.iter('email'):
            email = node.text
        for node in tree.iter('pub-date'):
            for subnode in node.iter('year'):
                collection = node.attrib
                #if "collection" in collection.values():
                if "pub-type" in collection.keys():
                    year = subnode.text
        for node in tree.iter('article-id'):
            pmidat = node.attrib
            if "pmc" in pmidat.values():
                pmid = node.text
            if "doi" in pmidat.values():
                doi = node.text
        
        for node in tree.iter('kwd'):  
            kwd = node.text
            if kwd != None:
                kwds.append(json.dumps(kwd))  
        kwdstring = str(kwds)
        kwdstring = kwdstring.replace("'", "")
        kwdstring = kwdstring.replace("\\", "")
                
        for node in tree.iter():
            if node.text != None:
                node = node.text.split()
                for rsnumber in node:
                    if rsnumber.isalnum() and len(rsnumber) > 4 and len(rsnumber) < 15: #ensure that digits follow rs
                        if rsnumber.startswith("rs") and rsnumber[2].isdigit():
                            while not rsnumber[-1].isdigit():
                                rsnumber = rsnumber[:-1] #ensure that rs ends with digits
                            rslist.append(rsnumber.strip() + "\t" + email.strip() + "\t" + year.strip() + \
                            "\t" + pmid.strip() + "\t" + doi.strip() + "\t" + file.strip() + "\t" + str(kwdstring))
rsdict = {}
for item in rslist:
    if rsdict.get(item,"empty") == "empty":
        rsdict.update({item:1})
    if rsdict.get(item,"empty") != "empty":
        rsdict[item] += 1
rslist = []
writetofile = open("/home/werner/Desktop/TEXTdb.csv", "a")
for item in rsdict.keys():
    writetofile.write(item + "\t" + str(rsdict[item]) + "\n") 
writetofile.close()

In [13]:
#Convert database to JSON
import json
file = open("/home/werner/Desktop/TEXTdb.csv", "r")
preJSONlist = []
JSONstring = "{\"PMCOAI_rs_articles\": ["
for line in file:
    lister = line.strip().split("\t")
    rs_number = "\"rs_number\": " + "\"" + lister[0] + "\""
    email_address = "\"email_address\": " + "\"" + lister[1] + "\""
    publication_date = "\"publication_date\": " + "\"" + lister[2] + "\""
    pubmed_id = "\"pubmed_id\": " + "\"" + lister[3] + "\""
    doi = "\"doi\": " + "\"" + lister[4] + "\""
    pubmed_file_name = "\"pubmed_file_name\": " + "\"" + lister[5] + "\""
    keywords = "\"keywords\": " + lister[6]
    rs_number_cited_in_article = "\"rs_number_cited_in_article\": " + "\"" + lister[7] + "\""
    newdictionary = "{" + rs_number + ", " + email_address + ", " + publication_date + ", " + pubmed_id + \
    ", " + doi + ", " + pubmed_file_name + ", " + rs_number_cited_in_article + ", " + keywords + "}"
    JSONstring += newdictionary + ", "
file.close()
with open("/home/werner/Desktop/JSONdbKeys.json", "w") as file1:
    JSONstring = JSONstring[:-2]
    JSONstring += "]}"
    file1.write(JSONstring)

In [4]:
#Test JSON
import json
from pprint import pprint
counter = 0
with open("/home/werner/Desktop/JSONdbKeys.json") as data_file:    
    data = json.load(data_file)
    for objs in data:
        counter += 1

z = list(data.values())
pprint(len(z[0]))

102150


In [11]:
#Get keywords
import json
from pprint import pprint
keywordList = []
with open("/home/werner/Desktop/JSONdbKeys.json") as data_file:    
    data = json.load(data_file)
    for i in range(len(data["PMCOAI_rs_articles"])):
        if len(data["PMCOAI_rs_articles"][i]["keywords"]) > 0:
            for keywords in data["PMCOAI_rs_articles"][i]["keywords"]:
                keywords = keywords.strip()
                if len(keywords) > 0 and all(x.isalnum() or x.isspace() for x in keywords) \
                and not keywords.startswith("rs"):
                    keywordList.append(keywords)
    keywordList = set(keywordList)
    keywordList = sorted(keywordList)
#print(len(json.dumps(keywordList)))
print(len(keywordList))

7713


In [14]:
#Get rs numbers
import json
from pprint import pprint
keywordList = []
with open("/home/werner/Desktop/JSONdbKeys.json") as data_file:    
    data = json.load(data_file)
    for i in range(len(data["PMCOAI_rs_articles"])):
        keywordList.append(data["PMCOAI_rs_articles"][i]["rs_number"])
    keywordList = set(keywordList)
    keywordList = sorted(keywordList)
with open("/home/werner/Desktop/rsnumbers.js", "w") as data_file: 
    data_file.write(json.dumps(keywordList))
print(len(keywordList))

60595
